In [ ]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

# Data import and preprocessing

this code allows to load dataset from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
airports = pd.read_csv("/content/drive/MyDrive/DataScience/airports.csv")

In [ ]:
flights = pd.read_csv("/content/drive/MyDrive/DataScience/flights.csv", low_memory=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
airlines = pd.read_csv("/content/drive/MyDrive/DataScience/airlines.csv")

## Basic dataset analysis

Dataset short analysis
- we would like to see the shape of the datasets and do some context review

In [ ]:
flights.shape

(5819079, 31)

In [ ]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flights.sample(10) # to look forward how the data looks

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4235116,2015,9,21,1,DL,1463,N925DL,CHS,ATL,1245,...,1345.0,-25.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2689404,2015,6,19,5,F9,566,N941FR,RDU,DEN,600,...,801.0,21.0,0,0,NaN,4.0,0.0,17.0,0.0,0.0
5384764,2015,12,3,4,WN,2149,N7819A,SNA,LAS,1830,...,1938.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
951474,2015,3,4,3,UA,673,N848UA,LGA,ORD,1000,...,1159.0,18.0,0,0,NaN,18.0,0.0,0.0,0.0,0.0
4079575,2015,9,11,5,EV,5087,N884AS,ATL,MLU,1035,...,1101.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
585849,2015,2,8,7,US,695,N151UW,PHX,LAX,1805,...,1837.0,6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3988787,2015,9,5,6,DL,2618,N392DA,RDU,LAX,726,...,1022.0,42.0,0,0,NaN,0.0,0.0,42.0,0.0,0.0
1971689,2015,5,6,3,AA,1299,N3LXAA,ORD,BOS,840,...,1142.0,-20.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4299254,2015,9,25,5,DL,1248,N973DL,ATL,CHS,1219,...,1315.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4814027,2015,10,28,3,DL,1925,N918DE,10397,10821,725,...,910.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flights.tail()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819078,2015,12,31,4,B6,839,N534JB,JFK,BQN,2359,...,442.0,2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

We can summarize that flights dataset contains information from 2015 about flights including data (split into separated columns: year, month, day), airline, origin and destination airports, shedule and real departure and destination time, elapsed time and air time, distance, information about wheels on/off or couse of the delay. 

##Data preparation

In [ ]:
# Deletion of duplicates
flights = flights.drop_duplicates()

flights.shape

# it was: (5819079, 31) so there was no duplicates

(5819079, 31)

Checking for missing values in percent

In [ ]:
percent_missing = flights.isnull().sum() * 100 / len(flights)
missing_value_flights = pd.DataFrame({'percent_missing': percent_missing})

In [ ]:
missing_value_flights

,percent_missing
YEAR,0.000000
MONTH,0.000000
DAY,0.000000
DAY_OF_WEEK,0.000000
AIRLINE,0.000000
FLIGHT_NUMBER,0.000000
TAIL_NUMBER,0.252978
ORIGIN_AIRPORT,0.000000
DESTINATION_AIRPORT,0.000000
SCHEDULED_DEPARTURE,0.000000


Most of missing data is in columns with a reason that could cause the delay.

Filling missing values with 0 in columns we will need (0 indicates there was no delay due to problems stated in column name)

In [ ]:
flights['AIR_SYSTEM_DELAY']=flights['AIR_SYSTEM_DELAY'].fillna(0)
flights['SECURITY_DELAY']=flights['SECURITY_DELAY'].fillna(0)
flights['AIRLINE_DELAY']=flights['AIRLINE_DELAY'].fillna(0)
flights['LATE_AIRCRAFT_DELAY']=flights['LATE_AIRCRAFT_DELAY'].fillna(0)
flights['WEATHER_DELAY']=flights['WEATHER_DELAY'].fillna(0)

Droping empty rows in arrival and departure time columns (less than 2% of all values)

In [ ]:
flights = flights[flights['DEPARTURE_DELAY'].notna()]
flights = flights[flights['DEPARTURE_TIME'].notna()]
flights = flights[flights['ARRIVAL_TIME'].notna()]
flights = flights[flights['ARRIVAL_DELAY'].notna()]

In [ ]:
# checking of data loss
flights.shape

(5714008, 31)

Next, we can see that the date is split into 3 separated columns: YEAR, MONTH, DAY. We convert columns YEAR, MONTH and DAY into one column DATE

In [ ]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,-22.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,5.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,-21.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,-26.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,-16.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,-8.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,-10.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31


Next, we can see that time is in format HHMM as float. We convert HHMM time format into datetime

In [ ]:
# Function that convert the 'HHMM' string to datetime.time
def format_hour(hour):
    if pd.isnull(hour):
        return np.nan
    else:
        if hour == 2400: 
          hour = 0
        hour = "{0:04d}".format(int(hour))
        formated_hour = datetime.time(int(hour[0:2]), int(hour[2:4]))
        return formated_hour

In [ ]:
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_hour)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_hour)

flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_hour)
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_hour)


In [ ]:
flights.dtypes

YEAR                            int64
MONTH                           int64
DAY                             int64
DAY_OF_WEEK                     int64
AIRLINE                        object
FLIGHT_NUMBER                   int64
TAIL_NUMBER                    object
ORIGIN_AIRPORT                 object
DESTINATION_AIRPORT            object
SCHEDULED_DEPARTURE            object
DEPARTURE_TIME                 object
DEPARTURE_DELAY               float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
SCHEDULED_TIME                float64
ELAPSED_TIME                  float64
AIR_TIME                      float64
DISTANCE                        int64
WHEELS_ON                     float64
TAXI_IN                       float64
SCHEDULED_ARRIVAL              object
ARRIVAL_TIME                   object
ARRIVAL_DELAY                 float64
DIVERTED                        int64
CANCELLED                       int64
CANCELLATION_REASON            object
AIR_SYSTEM_D

In [ ]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,00:05:00,...,-22.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,...,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,00:20:00,...,5.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,00:20:00,...,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,00:25:00,...,-21.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,23:59:00,...,-26.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,23:59:00,...,-16.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,23:59:00,...,-8.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,23:59:00,...,-10.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,2015-12-31


In [ ]:
# Export of prepared dataframe as csv

flights.to_csv('/content/drive/MyDrive/DataScience/Projekt końcowy/flights_prep1.csv')

Droping unnecessary columns

In [ ]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'DEPARTURE_TIME', 'YEAR', 'DAY']
flights.drop(variables_to_remove, axis = 1, inplace = True)
flights[:5]

,MONTH,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,1,4,AS,98,N407AS,ANC,SEA,00:05:00,-11.0,169.0,1448,04:30:00,-22.0,0.0,0.0,0.0,0.0,0.0,2015-01-01
1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,-8.0,263.0,2330,07:50:00,-9.0,0.0,0.0,0.0,0.0,0.0,2015-01-01
2,1,4,US,840,N171US,SFO,CLT,00:20:00,-2.0,266.0,2296,08:06:00,5.0,0.0,0.0,0.0,0.0,0.0,2015-01-01
3,1,4,AA,258,N3HYAA,LAX,MIA,00:20:00,-5.0,258.0,2342,08:05:00,-9.0,0.0,0.0,0.0,0.0,0.0,2015-01-01
4,1,4,AS,135,N527AS,SEA,ANC,00:25:00,-1.0,199.0,1448,03:20:00,-21.0,0.0,0.0,0.0,0.0,0.0,2015-01-01


##Replacing shortcuts of airlines and airports with full names



In [ ]:
flights.AIRLINE.unique()

array(['AS', 'AA', 'US', 'DL', 'NK', 'UA', 'HA', 'B6', 'OO', 'EV', 'F9',
       'WN', 'MQ', 'VX'], dtype=object)

In [ ]:
airlines[::]

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [ ]:
flights['AIRLINE'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'AA':'American Airlines',
    'DL':'Delta Airlines',
    'US':'American Airlines'
}, inplace=True)

In [ ]:
flights.AIRLINE.value_counts()

Southwest Airlines    1242403
American Airlines      907158
Delta Airlines         870275
SkyWest Airlines       576814
ExpressJet             554752
United Airlines        507762
Envoy Air              278791
JetBlue Airways        262042
Alaska Airlines        171439
Spirit Airlines        115193
Frontier Airlines       90090
Hawaiian Airlines       76041
Virgin America          61248
Name: AIRLINE, dtype: int64

In [ ]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [ ]:
airports.shape

(322, 7)

In [ ]:
airports = airports[['IATA_CODE','AIRPORT']]

airports

,IATA_CODE,AIRPORT
0,ABE,Lehigh Valley International Airport
1,ABI,Abilene Regional Airport
2,ABQ,Albuquerque International Sunport
3,ABR,Aberdeen Regional Airport
4,ABY,Southwest Georgia Regional Airport
...,...,...
317,WRG,Wrangell Airport
318,WYS,Westerly State Airport
319,XNA,Northwest Arkansas Regional Airport
320,YAK,Yakutat Airport


In [ ]:
airport_dic = dict(zip(airports.IATA_CODE, airports.AIRPORT))

airport_dic

{'ABE': 'Lehigh Valley International Airport',
 'ABI': 'Abilene Regional Airport',
 'ABQ': 'Albuquerque International Sunport',
 'ABR': 'Aberdeen Regional Airport',
 'ABY': 'Southwest Georgia Regional Airport',
 'ACK': 'Nantucket Memorial Airport',
 'ACT': 'Waco Regional Airport',
 'ACV': 'Arcata Airport',
 'ACY': 'Atlantic City International Airport',
 'ADK': 'Adak Airport',
 'ADQ': 'Kodiak Airport',
 'AEX': 'Alexandria International Airport',
 'AGS': 'Augusta Regional Airport\xa0(Bush Field)',
 'AKN': 'King Salmon Airport',
 'ALB': 'Albany International Airport',
 'ALO': 'Waterloo Regional Airport',
 'AMA': 'Rick Husband Amarillo International Airport',
 'ANC': 'Ted Stevens Anchorage International Airport',
 'APN': 'Alpena County Regional Airport',
 'ASE': 'Aspen-Pitkin County Airport',
 'ATL': 'Hartsfield-Jackson Atlanta International Airport',
 'ATW': 'Appleton International Airport',
 'AUS': 'Austin-Bergstrom International Airport',
 'AVL': 'Asheville Regional Airport',
 'AVP': 'W

In [ ]:
flights['DESTINATION_AIRPORT'].replace(airport_dic, inplace=True)
flights['ORIGIN_AIRPORT'].replace(airport_dic, inplace=True)
flights.sample(10)

,MONTH,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
3349991,7,2,American Airlines,1386,N434AA,Dallas/Fort Worth International Airport,Austin-Bergstrom International Airport,13:59:00,-5.0,33.0,190,15:01:00,-15.0,0.0,0.0,0.0,0.0,0.0,2015-07-28
2799472,6,4,ExpressJet,4666,N13969,Jackson-Evers International Airport,George Bush Intercontinental Airport,14:43:00,-8.0,59.0,351,16:04:00,-11.0,0.0,0.0,0.0,0.0,0.0,2015-06-25
4575141,10,2,SkyWest Airlines,4779,N461SW,11315,13487,05:30:00,-4.0,73.0,481,08:09:00,-16.0,0.0,0.0,0.0,0.0,0.0,2015-10-13
4886176,11,7,Spirit Airlines,971,N640NK,Fort Lauderdale-Hollywood International Airport,Dallas/Fort Worth International Airport,20:30:00,12.0,147.0,1119,22:31:00,3.0,0.0,0.0,0.0,0.0,0.0,2015-11-01
2686692,6,4,American Airlines,1851,N549UW,Charlotte Douglas International Airport,Chicago O'Hare International Airport,20:05:00,96.0,90.0,599,21:10:00,106.0,10.0,0.0,0.0,96.0,0.0,2015-06-18
1383219,3,1,Delta Airlines,726,N327NB,LaGuardia Airport (Marine Air Terminal),Minneapolis-Saint Paul International Airport,18:05:00,0.0,147.0,1020,20:22:00,-28.0,0.0,0.0,0.0,0.0,0.0,2015-03-30
3076222,7,7,American Airlines,473,N651AW,San Francisco International Airport,Phoenix Sky Harbor International Airport,11:55:00,-7.0,95.0,651,14:00:00,-19.0,0.0,0.0,0.0,0.0,0.0,2015-07-12
2001808,5,5,Frontier Airlines,1270,N228FR,Los Angeles International Airport,Chicago O'Hare International Airport,01:35:00,-14.0,202.0,1744,07:20:00,-20.0,0.0,0.0,0.0,0.0,0.0,2015-05-08
4944969,11,4,Southwest Airlines,181,N489WN,San Diego International Airport (Lindbergh Field),Denver International Airport,13:45:00,0.0,112.0,853,17:00:00,-6.0,0.0,0.0,0.0,0.0,0.0,2015-11-05
5361340,12,3,SkyWest Airlines,7377,N886AS,Salt Lake City International Airport,Bert Mooney Airport,11:05:00,-8.0,56.0,358,12:30:00,-15.0,0.0,0.0,0.0,0.0,0.0,2015-12-02


In [ ]:
# we have to check if all airports are changed with full name

flights.ORIGIN_AIRPORT.value_counts()

Hartsfield-Jackson Atlanta International Airport    343506
Chicago O'Hare International Airport                276554
Dallas/Fort Worth International Airport             232647
Denver International Airport                        193402
Los Angeles International Airport                   192003
                                                     ...  
14222                                                    1
14025                                                    1
13502                                                    1
15497                                                    1
12265                                                    1
Name: ORIGIN_AIRPORT, Length: 929, dtype: int64

In [ ]:
flights.DESTINATION_AIRPORT.value_counts()

Hartsfield-Jackson Atlanta International Airport    343076
Chicago O'Hare International Airport                275864
Dallas/Fort Worth International Airport             231764
Denver International Airport                        193033
Los Angeles International Airport                   192136
                                                     ...  
11097                                                    1
14025                                                    1
15497                                                    1
12265                                                    1
10666                                                    1
Name: DESTINATION_AIRPORT, Length: 929, dtype: int64

There are some rows with unidentified data in airpotrs column. We remove them

In [ ]:
airports_list = airports['AIRPORT'].to_list()
airports_list


['Lehigh Valley International Airport',
 'Abilene Regional Airport',
 'Albuquerque International Sunport',
 'Aberdeen Regional Airport',
 'Southwest Georgia Regional Airport',
 'Nantucket Memorial Airport',
 'Waco Regional Airport',
 'Arcata Airport',
 'Atlantic City International Airport',
 'Adak Airport',
 'Kodiak Airport',
 'Alexandria International Airport',
 'Augusta Regional Airport\xa0(Bush Field)',
 'King Salmon Airport',
 'Albany International Airport',
 'Waterloo Regional Airport',
 'Rick Husband Amarillo International Airport',
 'Ted Stevens Anchorage International Airport',
 'Alpena County Regional Airport',
 'Aspen-Pitkin County Airport',
 'Hartsfield-Jackson Atlanta International Airport',
 'Appleton International Airport',
 'Austin-Bergstrom International Airport',
 'Asheville Regional Airport',
 'Wilkes-Barre/Scranton International Airport',
 'Kalamazoo/Battle Creek International Airport',
 'Bradley International Airport',
 'Bethel Airport',
 'Meadows Field',
 'Greater 

In [ ]:
flights_new = flights[(flights.ORIGIN_AIRPORT.isin(airports_list) == True) & (flights.DESTINATION_AIRPORT.isin(airports_list) == True)] 
flights_new.ORIGIN_AIRPORT.value_counts()

Hartsfield-Jackson Atlanta International Airport    343506
Chicago O'Hare International Airport                276554
Dallas/Fort Worth International Airport             232647
Denver International Airport                        193402
Los Angeles International Airport                   192003
                                                     ...  
St. Cloud Regional Airport                              77
Dillingham Airport                                      77
Gustavus Airport                                        76
King Salmon Airport                                     63
Ithaca Tompkins Regional Airport                        30
Name: ORIGIN_AIRPORT, Length: 322, dtype: int64

In [ ]:
flights_new.DESTINATION_AIRPORT.value_counts()

Hartsfield-Jackson Atlanta International Airport    343076
Chicago O'Hare International Airport                275864
Dallas/Fort Worth International Airport             231764
Denver International Airport                        193033
Los Angeles International Airport                   192136
                                                     ...  
St. Cloud Regional Airport                              77
Dillingham Airport                                      77
Gustavus Airport                                        74
King Salmon Airport                                     63
Ithaca Tompkins Regional Airport                        31
Name: DESTINATION_AIRPORT, Length: 322, dtype: int64

In [ ]:
flights_new.shape

(5231130, 19)

In [ ]:
flights_new.sample(10)

,MONTH,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
3975651,9,5,United Airlines,306,N804UA,George Bush Intercontinental Airport,Gen. Edward Lawrence Logan International Airport,10:21:00,-1.0,201.0,1597,15:14:00,-2.0,0.0,0.0,0.0,0.0,0.0,2015-09-04
3789919,8,7,Hawaiian Airlines,224,N481HA,Lihue Airport,Honolulu International Airport,20:41:00,3.0,21.0,102,21:13:00,6.0,0.0,0.0,0.0,0.0,0.0,2015-08-23
3833874,8,3,SkyWest Airlines,5179,N784SK,George Bush Intercontinental Airport,City of Colorado Springs Municipal Airport,14:31:00,-5.0,116.0,809,15:58:00,-18.0,0.0,0.0,0.0,0.0,0.0,2015-08-26
3719925,8,3,United Airlines,1626,N68843,Seattle-Tacoma International Airport,George Bush Intercontinental Airport,13:51:00,-4.0,217.0,1874,20:02:00,-17.0,0.0,0.0,0.0,0.0,0.0,2015-08-19
2136752,5,6,Southwest Airlines,2832,N790SW,Cleveland Hopkins International Airport,Chicago Midway International Airport,10:40:00,0.0,49.0,307,10:50:00,-7.0,0.0,0.0,0.0,0.0,0.0,2015-05-16
3496677,8,4,Delta Airlines,1218,N3759,Detroit Metropolitan Airport,Ronald Reagan Washington National Airport,07:34:00,-3.0,61.0,405,08:59:00,-12.0,0.0,0.0,0.0,0.0,0.0,2015-08-06
2619059,6,7,Delta Airlines,776,N964DL,Hartsfield-Jackson Atlanta International Airport,Sarasota-Bradenton International Airport,22:17:00,18.0,62.0,444,23:50:00,12.0,0.0,0.0,0.0,0.0,0.0,2015-06-14
465726,1,6,Delta Airlines,2478,N935AT,Hartsfield-Jackson Atlanta International Airport,Jackson-Evers International Airport,14:56:00,-4.0,53.0,341,15:16:00,-15.0,0.0,0.0,0.0,0.0,0.0,2015-01-31
5479553,12,4,ExpressJet,5012,N868AS,Lehigh Valley International Airport,Detroit Metropolitan Airport,06:05:00,-7.0,82.0,425,07:53:00,-6.0,0.0,0.0,0.0,0.0,0.0,2015-12-10
3520458,8,5,American Airlines,1260,N548AA,Dallas/Fort Worth International Airport,Detroit Metropolitan Airport,13:42:00,23.0,130.0,986,17:30:00,4.0,0.0,0.0,0.0,0.0,0.0,2015-08-07


In [ ]:
flights_new.to_csv('/content/drive/MyDrive/DataScience/Projekt końcowy/new_flights.csv', index=False)